In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import random
import seaborn as sns
from itertools import product
import scipy.stats as stats
import re
import itertools
from scipy.stats import norm
from Step2.preprocessing.BA_params import *
from Step2.preprocessing.load_train_param import *
from Step2.preprocessing.filters import *
from Step2.LR.lr import *
from Step2.LR.trials import *
from Step2.LR.llr_prediction import *
from Step2.LR.performance import cllr, min_cllr
from sklearn.metrics.pairwise import cosine_similarity
import shap
import pickle

In [2]:
# path="/local_disk/clytie/ibenamor/phd_experiments/scripts/LIA/stklia/exemples/voxceleb_seuil/model_dir/xvectors/voxceleb1_nosil/xvectors.txt"
# path1="/home/jordan/Documents/Avignon M2/interpretabilité & explicabiloté/Projet Mr Bonas/tp_jef/tp_jef/vec_vox1.txt"
path="./vox1-transformed.txt"
#Load parameters
typ_path="./data/typ.txt"
dout_path="./data/dout.txt"


# with open("/home/jordan/Documents/Avignon M2/interpretabilité & explicabiloté/Projet Mr Bonas/BA-LR/data/test.txt", 'r') as infile, open('/home/jordan/Documents/Avignon M2/interpretabilité & explicabiloté/Projet Mr Bonas/BA-LR/data/test_transformed.txt', 'w') as outfile:
#      for line in infile:
#         parts = line.split()
#         # Extract the ID
#         id = parts[0]
# 
#         # Extract and concatenate vector parts
#         vector = ""
#         for part in parts[1:]:
#             vector += part
#             # Check if the vector part ends with ']'
#             if ']' in part:
#                 break
#             else:
#                 vector += " "
#         else:
#             # If ']' is not found, continue reading the next line
#             while True:
#                 next_line = next(infile).strip()
#                 vector += next_line
#                 if ']' in next_line:
#                     break
# 
#         # Remove extra spaces
#         vector = ' '.join(vector.split())
# 
#         # Write the transformed line to the output file
#         outfile.write(f"{id}   {vector}\n")
# 
# print("Transformation complete. Output saved to output_transformed.txt")

In [3]:
utterances, vectors = readVectors_test(path)
xvectors = np.array(vectors).astype('float64')
print(xvectors.shape)
BA = ['BA' + str(i) for i in range(xvectors.shape[1])]
df = pd.DataFrame(xvectors, columns=BA)
df=todelete(df,BA)
df
# que fait le code ci dessus?
# on a un dataframe avec les BA en colonnes et les vecteurs en lignes
# on supprime les BA qui ont une variance nulle
# on a donc un dataframe avec les BA en colonnes et les vecteurs en lignes
 

ligne id10270-5r0dWxy17C8-00002 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]

Progression : 0%
ligne id10270-5r0dWxy17C8-00011 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 

,BA0,BA1,BA2,BA3,BA4,BA5,BA6,BA7,BA8,BA9,...,BA196,BA197,BA198,BA199,BA200,BA201,BA202,BA203,BA204,BA205
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
377,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
378,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
379,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [4]:
utt_per_spk, loc_list = number_utterances(utterances)
utt_per_spk

speaker_dict {'id10270': 7, 'id10271': 4, 'id10272': 10, 'id10273': 17, 'id10274': 4, 'id10275': 5, 'id10276': 21, 'id10277': 7, 'id10278': 22, 'id10279': 2, 'id10280': 2, 'id10281': 7, 'id10282': 8, 'id10283': 19, 'id10284': 4, 'id10285': 4, 'id10286': 18, 'id10287': 1, 'id10289': 8, 'id10290': 7, 'id10291': 5, 'id10292': 31, 'id10293': 17, 'id10294': 7, 'id10295': 12, 'id10296': 9, 'id10297': 7, 'id10298': 8, 'id10300': 21, 'id10301': 2, 'id10302': 11, 'id10303': 5, 'id10304': 8, 'id10305': 10, 'id10306': 12, 'id10307': 14, 'id10308': 2, 'id10309': 23}
loc_list ['id10270', 'id10270', 'id10270', 'id10270', 'id10270', 'id10270', 'id10270', 'id10271', 'id10271', 'id10271', 'id10271', 'id10272', 'id10272', 'id10272', 'id10272', 'id10272', 'id10272', 'id10272', 'id10272', 'id10272', 'id10272', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10273', 'id10274', 

{'id10270': 7,
 'id10271': 4,
 'id10272': 10,
 'id10273': 17,
 'id10274': 4,
 'id10275': 5,
 'id10276': 21,
 'id10277': 7,
 'id10278': 22,
 'id10279': 2,
 'id10280': 2,
 'id10281': 7,
 'id10282': 8,
 'id10283': 19,
 'id10284': 4,
 'id10285': 4,
 'id10286': 18,
 'id10287': 1,
 'id10289': 8,
 'id10290': 7,
 'id10291': 5,
 'id10292': 31,
 'id10293': 17,
 'id10294': 7,
 'id10295': 12,
 'id10296': 9,
 'id10297': 7,
 'id10298': 8,
 'id10300': 21,
 'id10301': 2,
 'id10302': 11,
 'id10303': 5,
 'id10304': 8,
 'id10305': 10,
 'id10306': 12,
 'id10307': 14,
 'id10308': 2,
 'id10309': 23}

In [5]:
utt_spk = utterance_spk(utt_per_spk)
utt_spk

{'id10270': ['utt0', 'utt1', 'utt2', 'utt3', 'utt4', 'utt5', 'utt6'],
 'id10271': ['utt7', 'utt8', 'utt9', 'utt10'],
 'id10272': ['utt11',
  'utt12',
  'utt13',
  'utt14',
  'utt15',
  'utt16',
  'utt17',
  'utt18',
  'utt19',
  'utt20'],
 'id10273': ['utt21',
  'utt22',
  'utt23',
  'utt24',
  'utt25',
  'utt26',
  'utt27',
  'utt28',
  'utt29',
  'utt30',
  'utt31',
  'utt32',
  'utt33',
  'utt34',
  'utt35',
  'utt36',
  'utt37'],
 'id10274': ['utt38', 'utt39', 'utt40', 'utt41'],
 'id10275': ['utt42', 'utt43', 'utt44', 'utt45', 'utt46'],
 'id10276': ['utt47',
  'utt48',
  'utt49',
  'utt50',
  'utt51',
  'utt52',
  'utt53',
  'utt54',
  'utt55',
  'utt56',
  'utt57',
  'utt58',
  'utt59',
  'utt60',
  'utt61',
  'utt62',
  'utt63',
  'utt64',
  'utt65',
  'utt66',
  'utt67'],
 'id10277': ['utt68', 'utt69', 'utt70', 'utt71', 'utt72', 'utt73', 'utt74'],
 'id10278': ['utt75',
  'utt76',
  'utt77',
  'utt78',
  'utt79',
  'utt80',
  'utt81',
  'utt82',
  'utt83',
  'utt84',
  'utt85',
 

In [6]:
profil = profil_spk(xvectors, utt_per_spk, BA)
profil

{'id10270': {'BA0': 0,
  'BA1': 0,
  'BA2': 1,
  'BA3': 0,
  'BA4': 1,
  'BA5': 0,
  'BA6': 1,
  'BA7': 1,
  'BA8': 1,
  'BA9': 0,
  'BA10': 0,
  'BA11': 1,
  'BA12': 1,
  'BA13': 0,
  'BA14': 0,
  'BA15': 0,
  'BA16': 0,
  'BA17': 1,
  'BA18': 1,
  'BA19': 0,
  'BA20': 1,
  'BA21': 0,
  'BA22': 0,
  'BA23': 0,
  'BA24': 0,
  'BA25': 1,
  'BA26': 1,
  'BA27': 0,
  'BA28': 1,
  'BA29': 0,
  'BA30': 1,
  'BA31': 1,
  'BA32': 1,
  'BA33': 0,
  'BA34': 0,
  'BA35': 0,
  'BA36': 1,
  'BA37': 1,
  'BA38': 1,
  'BA39': 1,
  'BA40': 1,
  'BA41': 1,
  'BA42': 1,
  'BA43': 1,
  'BA44': 0,
  'BA45': 0,
  'BA46': 1,
  'BA47': 0,
  'BA48': 1,
  'BA49': 0,
  'BA50': 0,
  'BA51': 1,
  'BA52': 1,
  'BA53': 1,
  'BA54': 1,
  'BA55': 0,
  'BA56': 0,
  'BA57': 1,
  'BA58': 1,
  'BA59': 1,
  'BA60': 1,
  'BA61': 1,
  'BA62': 1,
  'BA63': 1,
  'BA64': 1,
  'BA65': 0,
  'BA66': 1,
  'BA67': 1,
  'BA68': 0,
  'BA69': 0,
  'BA70': 1,
  'BA71': 0,
  'BA72': 0,
  'BA73': 1,
  'BA74': 1,
  'BA75': 1,
  'BA76': 1

In [7]:
logging.info("computing combinations...")
couples = list(itertools.combinations(utt_per_spk.keys(), 2))
couples

[('id10270', 'id10271'),
 ('id10270', 'id10272'),
 ('id10270', 'id10273'),
 ('id10270', 'id10274'),
 ('id10270', 'id10275'),
 ('id10270', 'id10276'),
 ('id10270', 'id10277'),
 ('id10270', 'id10278'),
 ('id10270', 'id10279'),
 ('id10270', 'id10280'),
 ('id10270', 'id10281'),
 ('id10270', 'id10282'),
 ('id10270', 'id10283'),
 ('id10270', 'id10284'),
 ('id10270', 'id10285'),
 ('id10270', 'id10286'),
 ('id10270', 'id10287'),
 ('id10270', 'id10289'),
 ('id10270', 'id10290'),
 ('id10270', 'id10291'),
 ('id10270', 'id10292'),
 ('id10270', 'id10293'),
 ('id10270', 'id10294'),
 ('id10270', 'id10295'),
 ('id10270', 'id10296'),
 ('id10270', 'id10297'),
 ('id10270', 'id10298'),
 ('id10270', 'id10300'),
 ('id10270', 'id10301'),
 ('id10270', 'id10302'),
 ('id10270', 'id10303'),
 ('id10270', 'id10304'),
 ('id10270', 'id10305'),
 ('id10270', 'id10306'),
 ('id10270', 'id10307'),
 ('id10270', 'id10308'),
 ('id10270', 'id10309'),
 ('id10271', 'id10272'),
 ('id10271', 'id10273'),
 ('id10271', 'id10274'),


In [9]:

print("profil : ", profil)
print("couples : ", couples)
print("utt_spk : ", utt_spk)
print("BA : ", BA)
print("xvectors : ", xvectors)
print("typ_path : ", typ_path)
print("dout_path : ", dout_path)

typicality_and_dropout(profil, couples, utt_spk, BA, xvectors, typ_path, dout_path)



_ncouples_active_b :  171



 typ :  171



 dropout :  0.3621087950970945



 nb_BA_spk_b :  {'id10270': 0, 'id10271': 4, 'id10272': 2, 'id10273': 0, 'id10274': 0, 'id10275': 3, 'id10276': 5, 'id10277': 0, 'id10278': 3, 'id10279': 0, 'id10280': 0, 'id10281': 0, 'id10282': 0, 'id10283': 17, 'id10284': 0, 'id10285': 0, 'id10286': 18, 'id10287': 0, 'id10289': 0, 'id10290': 5, 'id10291': 0, 'id10292': 22, 'id10293': 2, 'id10294': 0, 'id10295': 11, 'id10296': 8, 'id10297': 7, 'id10298': 2, 'id10300': 0, 'id10301': 0, 'id10302': 0, 'id10303': 1, 'id10304': 5, 'id10305': 7, 'id10306': 0, 'id10307': 13, 'id10308': 0, 'id10309': 23}



 dropout_per_spk :  {'id10270': 0.0, 'id10271': 0.0, 'id10272': 0.8, 'id10273': 0.0, 'id10274': 0.0, 'id10275': 0.4, 'id10276': 0.7619047619047619, 'id10277': 0.0, 'id10278': 0.8636363636363636, 'id10279': 0.0, 'id10280': 0.0, 'id10281': 0.0, 'id10282': 0.0, 'id10283': 0.10526315789473684, 'id10284': 0.0, 'id10285': 0.0, 'id10286': 0.0, 'id10287': 0.0, 'id1028

ZeroDivisionError: float division by zero

In [ ]:
typ,dout=load_filter_soft(typ_path,dout_path)
df=df[list(typ.keys())]
BA_test=list(typ.keys())
# on charge les parametres typ et dout
# on garde les BA qui ont un typ et un dout non nuls
# on a donc un dataframe avec les BA en colonnes et les vecteurs en lignes

In [ ]:
utt={}
for (idx,row) in df.iterrows():
    utt[f"utt{idx}"]=dict(row)
# que fait le code ci dessus?
# on cree un dictionnaire avec les BA en cle et les vecteurs en valeurs

In [ ]:
non,tar= load_trials()
print("dout : ", dout)
LLR_target,LLR_non,list_eer,list_cllr_min,list_cllr_act,list_Din=LR_framework(dout,typ,utt,tar,non,[0.12])
plt.show()

# on charge les trials
# on calcule les LLR pour les trials
# on affiche les courbes eer, cllr_min, cllr_act, Din
# a quoi correspondent les trials?
#  les trials sont des paires d'indices d'utterances (i1,i2), qui correspond a une paire de vecteurs (v1,v2) avec v1=utt[i1] et v2=utt[i2]

# a quoi correspond LLR_target, LLR_non ?: LLR_target: LLR pour les trials target, LLR_non: LLR pour les trials non target. on a besoin de ça pour calculer les statistiques pour les LLR partiels (voir plus bas).
# a quoi correspond eer, cllr_min, cllr_act, Din?
# eer: equal error rate:  taux d'erreur pour lequel le taux de faux positifs est egal au taux de faux negatifs
# cllr_min: cllr minimal: cllr pour lequel le taux de faux positifs est egal au taux de faux negatifs
# cllr_act: cllr actuel: cllr pour lequel le taux de faux positifs est egal au taux de faux negatifs
# Din: detection cost: cout de detection

ZeroDivisionError: division by zero

In [ ]:
utt_llr_tar,llr_type_tar=partial_lr_analysis(tar, BA_test,utt,typ, dout, prob_dropin=0.12)
utt_llr_non,llr_type_non=partial_lr_analysis(non,BA_test,utt,typ, dout, prob_dropin=0.12)
df_llr_tar=stats(utt_llr_tar,LLR_target)
df_llr_non=stats(utt_llr_non,LLR_non)
# que fait le code ci dessus?
# on calcule les LLR partiels pour les trials
# on calcule les statistiques pour les LLR partiels

KeyError: 'utt116816'

In [ ]:
BA_tar={}
for ba in BA_test:
    ba_target=[]
    for i1,i2 in tar:
        ba_target.append((int(utt[i1][ba]),int(utt[i2][ba])))
    BA_tar[ba]=ba_target
#composition of each utterance in the non couples
BA_non={}
for ba in BA_test:
    ba_non=[]
    for i1,i2 in non:
        ba_non.append((int(utt[i1][ba]),int(utt[i2][ba])))
    BA_non[ba]=ba_non
df_BA_non=pd.DataFrame(BA_non)
df_non=pd.DataFrame(pd.Series(non),columns=["non"])
df_non=pd.concat([df_non,df_BA_non],axis=1)
df_BA=pd.DataFrame(BA_tar)
df_tar=pd.DataFrame(pd.Series(tar),columns=["target"])
df_tar=pd.concat([df_tar,df_BA],axis=1)
df_tar["scores"]=LLR_target
df_non["scores"]=LLR_non
# que fait le code ci dessus?
# on cree un dataframe avec les trials en lignes et les BA en colonnes

In [ ]:
scores_BA={}
for b in BA_test:
    if b in BA_test:
        scores_BA[b]={"00":0,"01":0,"10":0,"11":0}
        for i in set(df_llr_tar[b]):
            index=df_llr_tar[df_llr_tar[b]==i].index[0]
            if df_tar[b].loc[index]==(0, 0):
                scores_BA[b]["00"]=i
            elif df_tar[b].loc[index]==(1, 0):
                scores_BA[b]["10"]=i
            elif df_tar[b].loc[index]==(0, 1):
                scores_BA[b]["01"]=i
            else:
                scores_BA[b]["11"]=i
# que fait le code ci dessus?
# on cree un dictionnaire avec les BA en cle et les LLR partiels en valeurs

In [ ]:
scores_df=pd.DataFrame.from_dict(scores_BA).T
plt.figure(figsize=(4,3))
sns.boxplot(data=scores_df, orient="v")
plt.title("Distribution of partial LLRs")
plt.xlabel("LLRs")
plt.ylabel("Partial LLRs")
plt.show()

In [ ]:
scores_BA_df=pd.DataFrame()
scores_BA_df["BA"]=typ.keys()
scores_BA_df["Typicality"]=typ.values()
scores_BA_df["Dropout"]=dout.values()
scores_BA_df=scores_BA_df.sort_values("Typicality")
# que fait le code ci dessus?
# on cree un dataframe avec les BA en lignes et les parametres typ et dout en colonnes
# on trie les BA par ordre croissant de typicalite
# a quoi correspond les parametres typ et dout?
# typ: typicality: typicite: probabilité qu'un BA soit present dans un vecteur
# dout: dropout: probabilité qu'un BA soit absent dans un vecteur

In [ ]:
scores_BA_df=scores_BA_df.set_index('BA')
scores_BA_df=pd.concat([scores_BA_df,scores_df],axis=1)
scores_BA_df.reset_index(inplace=True)


In [ ]:
tar=[i for i in list(df_tar["target"])]
non=[i for i in list(df_non["non"])]
def data_shape(tar,non,typ,dout,utt):
    U1=[]
    U2=[]
    for i in tar+non:
        c1=[]
        #print(utt[i[0]])
        for j in utt[i[0]].keys():
            c1.append(str(int(utt[i[0]][j]))+str(int(utt[i[1]][j])))
        U1.append(c1)
    test1=pd.DataFrame(U1,columns=BA_test)
    test1=test1.replace("01",1)
    test1=test1.replace("10",1)
    test1=test1.replace("00",2)
    test1=test1.replace("11",3)
    for i in test1.columns:
        test1[i] = test1[i].astype("category")
    return test1

# que fait le code ci dessus?
# on cree un dataframe avec les BA en colonnes et les trials en lignes
# le lr est calculé pour chaque BA
def model(test):
    llr=[]
    test=pd.DataFrame(test,columns=BA_test)
    for idx,row in test.iterrows():
        lr=1
        for i in BA_test:
            #print(test.loc[idx][i])
            if test.loc[idx][i]!=0:
                if test.loc[idx][i]==3:
                    lr*=LR_11(typ[i],dout[i],prob_dropin)
                elif test.loc[idx][i]==2:
                    lr *= LR_00(typ[i], dout[i], prob_dropin)
                else:
                    lr *= LR_01(typ[i], prob_dropin)
        llr.append(np.log(lr))
    return np.array(llr)

# que fait le code ci dessus?
# on cree une fonction qui calcule les LLR partiels pour un vecteur

def custom_masker(mask, x):
    
    return (x * mask).reshape(1,len(x))

# on cree une fonction qui applique un masque a un vecteur
# on applique le masque a un vecteur pour avoir un vecteur avec des 0; pourquoi ? : pour avoir un vecteur avec des 0 et des 1

In [ ]:
prob_dropin=0.12
explainer = shap.Explainer(model,custom_masker)


In [ ]:
test=data_shape(tar[:2],non[:2],typ,dout,utt)

# que fait le code ci dessus, test=data_shape(tar[:2],non[:2],typ,dout,utt)?
# on cree un dataframe avec les BA en colonnes et les vecteurs en lignes
# on prend les deux premiers trials
# on calcule les LLR partiels pour les deux premiers trials
# on cree un dataframe avec les LLR partiels en colonnes et les trials en lignes
# a quoi corresponnd un trial?: une paire d'indices d'utterances
shape=explainer(test)
test1=data_shape(tar,non,typ,dout,utt)

In [ ]:
shape.values=np.concatenate((np.array(df_llr_tar[BA_test]),np.array(df_llr_non[BA_test])),axis=0)
shape.base_values=np.array([0.]*153516)
shape.data=np.array(test1)

## Local explanation

In [ ]:
import shap
shap.initjs()
shap.plots.beeswarm(shape,206)

In [ ]:
shap.initjs()
#prediction number 0
shap.plots.force(shape[0])

In [ ]:
shap.plots.waterfall(shape[-21])

## Global explanation

In [ ]:
contribution={}
for b in BA_test:
    contribution[b]=round((np.sum(np.abs(df_llr_tar[b]))+np.sum(np.abs(df_llr_non[b])))/(df_llr_non.shape[0]+df_llr_tar.shape[0]),3)
contribution_dict={k: (v/sum(contribution.values()))*100 for k, v in sorted(contribution.items(), key=lambda item: item[1],reverse=True)}
contribution_dict

In [ ]:
t=[]
d=[]
for i in contribution_dict.keys():
    t.append(typ[i])
    d.append(dout[i])
df_scores=pd.DataFrame([contribution_dict])
df_scores=df_scores.T
df_scores.rename(columns = {0:'Contribution'}, inplace = True)
df_scores["typicality"]=t
df_scores['dropout']=d
df_scores["BA"]=contribution_dict.keys()
df_scores

In [ ]:
import plotly.express as px

fig=px.scatter(df_scores, x="typicality", y="Contribution", color="dropout",title="Individual BA contribution Vs typicality & dropout ",width=500, height=400)
fig.show()